In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [2]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        #rmse = mean_squared_error(y, y_pred=sr)**0.5
        rmse = librosa.feature.rms(y=y)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00093.wav,0.377690,0.065906,569.930721,995.407125,927.427725,0.021701,-350.436188,169.545746,31.820370,...,1.821690,-5.970891,-5.259567,-0.229211,-1.776850,-3.713751,0.181591,2.072390,-2.896225,blues
1,blues.00087.wav,0.336773,0.158098,1442.190271,1870.534155,3083.414688,0.050889,-155.504929,125.638863,1.596553,...,-0.792893,-7.748057,0.413548,-7.030263,3.997679,-6.256611,0.958227,2.019821,-5.742188,blues
2,blues.00050.wav,0.400860,0.182380,1945.848425,2082.246626,4175.874749,0.085806,-82.979019,107.052124,-25.320452,...,12.539581,-9.762303,2.562253,-6.300853,2.996785,-8.718455,-0.326581,-2.980347,0.712600,blues
3,blues.00044.wav,0.390212,0.136276,2279.124558,2375.102120,5198.360233,0.092570,-109.509285,86.922409,-8.607986,...,11.087481,-5.085794,3.976360,-12.859742,12.343859,0.026216,-0.741568,-5.126620,3.303442,blues
4,blues.00078.wav,0.414188,0.258052,2333.685108,2227.425609,4942.811778,0.123863,-2.524338,101.252716,-33.924385,...,12.506608,-13.368822,6.112817,-9.065890,5.033774,-11.330277,3.166534,-4.567591,-4.033623,blues


In [6]:
data.shape

(1000, 28)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[10]

array([-0.81443786, -1.51414392, -1.18146635, -1.42150444, -1.31829652,
       -0.6403677 , -1.38688724,  1.91690394, -0.95086388, -1.0649626 ,
       -0.71555865, -1.52585945, -0.58998316, -1.90225691, -0.15705653,
       -1.50964074,  0.46353713, -1.55585341,  0.12566707, -1.02457131,
        0.09925202, -0.65706693,  0.77509555,  0.35974649,  0.58946277,
       -0.44894757])

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 0s 2ms/step - loss: 2.2039 - accuracy: 0.2062
Epoch 2/20
7/7 [==============================] - 0s 2ms/step - loss: 1.8941 - accuracy: 0.3487
Epoch 3/20
7/7 [==============================] - 0s 2ms/step - loss: 1.6548 - accuracy: 0.4325
Epoch 4/20
7/7 [==============================] - 0s 2ms/step - loss: 1.4622 - accuracy: 0.4875
Epoch 5/20
7/7 [==============================] - 0s 2ms/step - loss: 1.3179 - accuracy: 0.5600
Epoch 6/20
7/7 [==============================] - 0s 2ms/step - loss: 1.2017 - accuracy: 0.6075
Epoch 7/20
7/7 [==============================] - 0s 2ms/step - loss: 1.1140 - accuracy: 0.6313
Epoch 8/20
7/7 [==============================] - 0s 2ms/step - loss: 1.0310 - accuracy: 0.6800
Epoch 9/20
7/7 [==============================] - 0s 1ms/step - loss: 0.9655 - accuracy: 0.6963
Epoch 10/20
7/7 [==============================] - 0s 1ms/step - loss: 0.9024 - accuracy: 0.7163
Epoch 11/20
7/7 [======================

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 1ms/step - loss: 0.9565 - accuracy: 0.6700


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.6700000166893005


In [19]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [20]:

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 0s 69ms/step - loss: 2.2869 - accuracy: 0.1167 - val_loss: 2.1349 - val_accuracy: 0.3550
Epoch 2/30
2/2 [==============================] - 0s 11ms/step - loss: 2.1138 - accuracy: 0.3500 - val_loss: 2.0000 - val_accuracy: 0.3450
Epoch 3/30
2/2 [==============================] - 0s 12ms/step - loss: 1.9616 - accuracy: 0.3583 - val_loss: 1.8658 - val_accuracy: 0.3350
Epoch 4/30
2/2 [==============================] - 0s 13ms/step - loss: 1.8106 - accuracy: 0.3600 - val_loss: 1.7442 - val_accuracy: 0.3600
Epoch 5/30
2/2 [==============================] - 0s 12ms/step - loss: 1.6827 - accuracy: 0.3817 - val_loss: 1.6113 - val_accuracy: 0.3700
Epoch 6/30
2/2 [==============================] - 0s 12ms/step - loss: 1.5581 - accuracy: 0.4100 - val_loss: 1.4822 - val_accuracy: 0.4550
Epoch 7/30
2/2 [==============================] - 0s 12ms/step - loss: 1.4415 - accuracy: 0.4900 - val_loss: 1.3816 - val_accuracy: 0.5000
Epoch 8/30
2/2 [===========

In [21]:
results

[0.9713917374610901, 0.6349999904632568]

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

1.0

In [25]:
np.argmax(predictions[0])

2